# Ex5: Pole sharing
In vanilla vector fitting, all responses share a common set of poles. In the literature, this case is called MIMO fitting.

There are applications in which this can lead to problems and it is desirable to use a separate pole set for each response. In the literature, this is called Multi-SISO fitting.

It is also possible to go anywhere between those two extreme cases, for example by using a common set of poles for every input j, for example: S1j, S2j, S3j, ... will share one set of poles for every value of the input j. We thus have one pole set per input, which is used for the transfer of this input j to all outputs j=1, 2, 3, ... This is why in the literature this approach is called Multi-SIMO fitting.

In general, arbitrary combinations of inputs and outputs can be grouped to use a separate set of poles per group.

In vectorFitting, MIMO, Multi-SISO, Multi-SIMO, Multi-MISO and arbitrary custom pole groups are supported.

This example shows how those can be used.

## MIMO
The following example shows how to do a MIMO fit. This is the default if not explicitly specified. It will put all responses into one pole group, thus all responses will share one set of poles.

In [ ]:
import matplotlib.pyplot as mplt
import numpy as np

import skrf

nw = skrf.network.Network('./190ghz_tx_measured.S2P')
vf = skrf.VectorFitting(nw)
vf.vector_fit(n_poles_init=16, poles_init_type = 'real', pole_sharing='MIMO', verbose=True)

We can then plot the fitted responses:

In [ ]:
freqs = np.linspace(np.min(vf.network.f), np.max(vf.network.f), 201)
n_ports=vf._get_n_ports()
fig, ax = mplt.subplots(n_ports, n_ports)
fig.set_size_inches(12, 8)
for i in range(n_ports):
    for j in range(n_ports):
        vf.plot_s_db(i, j, freqs=freqs, ax=ax[i][j])
fig.tight_layout()
mplt.show()

## Multi-SISO
Now we run the fit as a Multi-SISO fit with the same total model order (16), thus every response gets a model order of 4:

In [ ]:
vf.vector_fit(n_poles_init=4, poles_init_type = 'real', pole_sharing='Multi-SISO', verbose=True)

As can be observed from the model summary, both the total absolute and relative errors are worse than in the MIMO fit with the same total model order. Thus, it is not beneficial to use Multi-SISO for this example because there are probably shared poles in the system that we are now treating as separate, which is not optimal.

Note that this example data is just used to demonstrate the process. If Multi-SISO fitting is advantageous or not depends on your data.

We can now plot the responses:

In [ ]:
freqs = np.linspace(np.min(vf.network.f), np.max(vf.network.f), 201)
n_ports=vf._get_n_ports()
fig, ax = mplt.subplots(n_ports, n_ports)
fig.set_size_inches(12, 8)
for i in range(n_ports):
    for j in range(n_ports):
        vf.plot_s_db(i, j, freqs=freqs, ax=ax[i][j])
fig.tight_layout()
mplt.show()

## Multi-SIMO
Now we run the fit as a Multi-SIMO fit with the same total model order (16). As we have two outputs, we will get two pole groups. To achieve the same total model order, we set n_poles_init=8

In [ ]:
vf.vector_fit(n_poles_init=8, poles_init_type = 'real', pole_sharing='Multi-SIMO', verbose=True)

In [ ]:
freqs = np.linspace(np.min(vf.network.f), np.max(vf.network.f), 201)
n_ports=vf._get_n_ports()
fig, ax = mplt.subplots(n_ports, n_ports)
fig.set_size_inches(12, 8)
for i in range(n_ports):
    for j in range(n_ports):
        vf.plot_s_db(i, j, freqs=freqs, ax=ax[i][j])
fig.tight_layout()
mplt.show()

## Multi-MISO
Now we run the fit as a Multi-MISO fit with the same total model order (16). As we have two inputs, we will get two pole groups. To achieve the same total model order, we set n_poles_init=8

In [ ]:
vf.vector_fit(n_poles_init=8, poles_init_type = 'real', pole_sharing='Multi-MISO', verbose=True)

We can already observe from the model summary how the pole groups are now composed differently than before.

In [ ]:
freqs = np.linspace(np.min(vf.network.f), np.max(vf.network.f), 201)
n_ports=vf._get_n_ports()
fig, ax = mplt.subplots(n_ports, n_ports)
fig.set_size_inches(12, 8)
for i in range(n_ports):
    for j in range(n_ports):
        vf.plot_s_db(i, j, freqs=freqs, ax=ax[i][j])
fig.tight_layout()
mplt.show()

## Custom pole groups
Custom pole groups can be created by specifying pole_sharing='custom' and providing the pole_groups
matrix that contains integers as a numpy 2-d array of shape (n_ports, n_ports).

If all integers are distinct, every response will go into its own pole group.
If some of the are equal, all of them will go into a common pole group.

Pole groups will be ordered such that the smallest integer in the matrix will represent the
first pole group and so on.

Example 1: To achieve the same effect as in pole_sharing='Multi-SIMO' for a 3 x 3 network,
you can provide the following pole_groups matrix:
pole_groups=np.array(([0, 1, 2, 3], [0, 1, 2, 3], [0, 1, 2, 3]))

Example 2: To put S11 and S13 into a separate pole group, and all other respones into
another pole group, for a 3 x 3 network, you can provide the following pole_groups matrix:
pole_groups=np.array(([0, 1, 0], [1, 1, 1], [1, 1, 1]))

In the following code example we will replicate the behavior of Multi-MISO that we did in the example before, but this time by specifying a custom pole group. This way we can verify that the result will be exactly the same as before:          

In [ ]:
vf.vector_fit(
    n_poles_init=8, poles_init_type = 'real', verbose=True,
    pole_sharing='custom', pole_groups=np.array(([1, 1], [2, 2]))
)

We can see from the model summary that we got exactly the same result. Fine, now for something more fancy. Let's put in the response (1, 2) into a separate pole group and all other responses into another:

In [ ]:
vf.vector_fit(
    n_poles_init=8, poles_init_type = 'real', verbose=True,
    pole_sharing='custom', pole_groups=np.array(([1, 2], [1, 1]))
)

We can see that now, as expected, the response (1, 2) got it's own pole set with 8 poles. In the plot below we can observe the effect. This response was fitted pretty good. But of course all other responses sharing the remaining 8 poles did not fit as good.

In [ ]:
freqs = np.linspace(np.min(vf.network.f), np.max(vf.network.f), 201)
n_ports=vf._get_n_ports()
fig, ax = mplt.subplots(n_ports, n_ports)
fig.set_size_inches(12, 8)
for i in range(n_ports):
    for j in range(n_ports):
        vf.plot_s_db(i, j, freqs=freqs, ax=ax[i][j])
fig.tight_layout()
mplt.show()

### Custom pole groups with custom initial poles
Up to now, we used a fixed number of n_poles_init=8 for all pole groups. But it is supported to specify a list of initial poles instead, so that we can specify n_poles_init differently for every pole group.

The same works for poles_init_type and poles_init_spacing.

Let's keep our 8 poles for the response (1, 2) but increase the number of initial poles for the other pole group to 16:

In [ ]:
vf.vector_fit(
    n_poles_init=[16, 8], poles_init_type = 'real', verbose=True,
    pole_sharing='custom', pole_groups=np.array(([1, 2], [1, 1]))
)

We can already see from the model summary that indeed we got 16 poles for group 1 and 8 for group 2, as desired.
Now let's check the plots:

In [ ]:
freqs = np.linspace(np.min(vf.network.f), np.max(vf.network.f), 201)
n_ports=vf._get_n_ports()
fig, ax = mplt.subplots(n_ports, n_ports)
fig.set_size_inches(12, 8)
for i in range(n_ports):
    for j in range(n_ports):
        vf.plot_s_db(i, j, freqs=freqs, ax=ax[i][j])
fig.tight_layout()
mplt.show()

We can see that now the responses from group 1 fit way better to the data than before.
To show the full power of this feature we now create 16 real poles in linear spacing for pole group 1 and 8 complex poles in logarithmic spacing for pole group 2. Of course the model order for pole group 2 will now also be 16 because we have complex conjugated pole pairs. 

In [ ]:
vf.vector_fit(
    n_poles_init=[16, 8], poles_init_type = ['real', 'complex'], poles_init_spacing=['lin', 'log'], verbose=True,
    pole_sharing='custom', pole_groups=np.array(([1, 2], [1, 1]))
)

Let's again plot the responses:

In [ ]:
freqs = np.linspace(np.min(vf.network.f), np.max(vf.network.f), 201)
n_ports=vf._get_n_ports()
fig, ax = mplt.subplots(n_ports, n_ports)
fig.set_size_inches(12, 8)
for i in range(n_ports):
    for j in range(n_ports):
        vf.plot_s_db(i, j, freqs=freqs, ax=ax[i][j])
fig.tight_layout()
mplt.show()

### Custom initial poles
If you already have a good idea where your initial poles will be, you can directly specify them instead of spacing a certain amount of them linearly or logarithmically over the frequency range:

In [ ]:
custom_initial_poles=vf.poles
vf.vector_fit(
    poles_init=custom_initial_poles, verbose=True,
    pole_sharing='custom', pole_groups=np.array(([1, 2], [1, 1]))
)

We can observe that the model order stays exactly the same because we used the output poles of the previous fit as new initial poles

If we specify poles_init not as a list but rather as a numpy array with poles, the same initial poles will be used for all pole groups:

In [ ]:
custom_initial_poles=vf.poles[0]
vf.vector_fit(
    poles_init=custom_initial_poles, verbose=True,
    pole_sharing='custom', pole_groups=np.array(([1, 2], [1, 1]))
)

That's it for this tutorial. Happy fitting!